<a href="https://colab.research.google.com/github/Flipeviotto/a_little_is_enough_attack/blob/Version_1/ataqueLiE_my_teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

import torch.nn as nn   # oferece um modulo de rede neural pré definido

import torch.optim as optim     # ajusta pesos e biases interativamente para minimizar um loss function
                                # exemplo de loss function: SGD, Adam, RMSprop

import torch.nn.functional as F # funções comuns para neur. netw. funções de ativação, pooling e loss functions.

from torch.utils.data import Subset     # pode criar subconjunto de dataset para validar e treinos especificos.

from torchvision.datasets import MNIST  # acessa o dataset MNIST

from torch.utils.data import DataLoader # carrega dados do dataset in batches, permite treinar e avaliar a rede neural

from torchvision.transforms import ToTensor # converte imagens para tensors

from sklearn.metrics import precision_recall_fscore_support # calcula as metricas da evolução


In [ ]:
class Arguments():  # encapsula hiperparametros e opções de configuração for a deep learning model
    def __init__(self):     # é um construtor dos atributos abaixo
        self.train_batch_size = 64
        self.test_batch_size = 64
        self.epochs = 20            # quantidade de epocas de treinamento
        self.learning_rate = 0.01
        self.no_cuda = False
        self.n_workers = 10

args = Arguments()
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
def get_mnist():
    return MNIST(root="./data", train=True, download=True, transform=ToTensor()), MNIST(root="./data", train=False, download=True, transform=ToTensor())

def prepare_dataset():
    trainset, testset = get_mnist()

    #Faz a divisão do dataset em partes para cada worker ter o seu próprio conjunto de dados
    trainsets = []
    parte_size = len(trainset) // args.n_workers z

    for i in range(args.n_workers):     # separa cada quantidade de dado em um conjunto de treinamento
        parte = Subset(trainset, range(i * parte_size, (i + 1) * parte_size))
        trainsets.append(parte)         # cada parte dessa lista será usada para treinar um worker

    if len(trainset) % args.n_workers != 0:     # se houver sobra de dados será adicionada na ultima parte
        parte_final = Subset(trainset, range(10 * parte_size, len(trainset)))
        trainsets.append(parte_final)

    #Cria um dataloader para cada worker
    trainloaders = []

    for trainset_ in trainsets:

        trainloaders.append(
            DataLoader(trainset_, batch_size=args.train_batch_size, shuffle=False)
        )

    testloaders = DataLoader(testset, batch_size=args.test_batch_size, shuffle=False)

    return trainloaders, testloaders


In [ ]:
class Net(nn.Module):                           # nn.Module define modulos de rede neural
    def __init__(self):                         # constroe a classe net
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)        # camada conv 2D, 1 canal de entrada e 20 de saida. kernel de 5
                                                            # essa camada serve para extrair caracteristicas das imagens de entrada
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)       # camada conv 2D, 20 canais de entrada e 50 de saida, kernel 5.
        self.fc1 = nn.Linear(50 * 4 * 4, 512)               # 50*4*4=800 é o tamanho da entrada da camada atual e da saida da camanda anterior.
            #fc é uma camada densa                          # tem 512 neuronio de saida
        self.fc2 = nn.Linear(512, 10)                       # tem 512 entradas e 10 saidas (é a quantidade de classes possiveis)

    def forward(self, x):                           # isso define a sequencia que os dados de entradas (x) serão submetidos para produzir a saida
        x = F.relu(self.conv1(x))                   # aplica uma camada de ativação em x e usa uma função de ativação para o resultado
        x = F.max_pool2d(x, 2)                      # aplica um kernel de tamanho 2 na entrada, o que reduz a dimenção espacial
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 50 * 4 * 4)
        x = F.relu(self.fc1(x))                     # é uma camada densa (fc)
        x = F.log_softmax(self.fc2(x), dim=1)       # log_softmax serve para normalizar a saida em probabilidades
        return x

In [ ]:
def train(args:Arguments, model:list, device, train_loader, optimizer, loss_fn, i):
    #treina o modelo local de cada worker apenas com o dataset remetente aquele worker
    loss = [0] * args.n_workers                     # inicializa as funções de perdas para cada worker

    # loop para cada worker
    for j in range(0, args.n_workers):
        model[j].train()                       # ajusta o modelo para modo de treinamento
        for _, (data, target) in enumerate(train_loader[j]):
            data, target = data.to(device), target.to(device)       # move os dados do target para o dispositivo escolhido (cpu ou gpu)
            optimizer[j].zero_grad()                                # limpa o gradiente do optimizer do worker para evitar que gradientes desatualizados afetem a atualização do parameto
            output = model[j](data)                                 # obtem a predição do worker atual
            loss[j] = loss_fn[j](output, target)                    # calcula a perda
            loss[j].backward()                                      # calcula o gradiente da perda
            optimizer[j].step()                                     # atualiza os parametros do modelo usado o optimizer

        print(f'Epoch {i} | Worker: {j} | TRAIN: Last Loss: {loss[j]:.5f}')


In [ ]:
def test(model, device, test_loader):
    model.eval()
    all_predictions = []
    all_targets = []
    correct = 0

    with torch.no_grad():
        for _, (data, target) in enumerate(test_loader):            # intera sobre cada dado e seu rotulo, _ descarta os index que não serão usados na função
            data, target = data.to(device), target.to(device)       # transfere os dados e os rotulos para o dispositivo escolhido
            outputs = model(data)                                   # gera as predictions
            loss = F.nll_loss(outputs, target).item()               # calcula o erro
            pred = outputs.argmax(1, keepdim=True)                  #
            correct += pred.eq(target.view_as(pred)).sum().item()   # calcula o numero predições corretas

            all_predictions.extend(pred.cpu().numpy().flatten())
            all_targets.extend(target.cpu().numpy().flatten())

        total_samples = len(test_loader.dataset)
        precision, recall, f1_score, _ = precision_recall_fscore_support(all_targets, all_predictions, average='weighted', zero_division=1)
        accuracy = (correct / total_samples) * 100

        with open(f"teste{wm}.txt", 'a') as file:
            file.write(f'Last Loss: {loss:.4f} | Accuracy: {accuracy:.2f}% | Correct: {correct}/{total_samples} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1-score: {f1_score:.4f}\n')
        print(f'Last Loss: {loss:.4f} | Accuracy: {accuracy:.2f}% | Correct: {correct}/{total_samples} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1-score: {f1_score:.4f}')


In [ ]:
def fedavg_aggregation(model:list, global_model):
    all_params = [list(model_agg.parameters()) for model_agg in model]              # intera pelos modelos de aprendizado federado treinados e retira os pesos e bias (parametros)
    average_params = [torch.stack(params).mean(0) for params in zip(*all_params)]   # empilha os parametros e calcula a media e amazena cada média em uma lista

    for own_params, avg_params in zip(global_model.parameters(), average_params):   # atualiza o modelo global,
        own_params.data.copy_(avg_params)

    for i in range(0, len(model)):                                                  #
        model[i].load_state_dict(global_model.state_dict())                         #

In [ ]:
from scipy.stats import norm

def ataque(args:Arguments, global_model:Net, model:list, workers_maliciosos:list):
    #nova tentativa
    total_clientes = args.n_workers
    #calcular a taxa de perturbação
    s = total_clientes/2 + 1 - len(workers_maliciosos)
    possibilidade = (total_clientes-s)/total_clientes
    z = norm.cdf(possibilidade)

    # somar parametros
    soma_parametros = Net().to(device)
    media = Net().to(device)
    desvio_padrao = Net().to(device)
    desvio_padrao_aux = Net().to(device)

    for parametro, parametro_media, desvio, desvio_pad in zip(soma_parametros.parameters(), media.parameters(), desvio_padrao_aux.parameters(), desvio_padrao.parameters()):
        parametro.data.zero_()
        parametro_media.data.zero_()
        desvio.data.zero_()
        desvio_pad.data.zero_()


    with torch.no_grad():
      for malicioso in workers_maliciosos:
        for parametro1, parametro2 in zip(model[malicioso].parameters(), soma_parametros.parameters()):
          parametro2.data += parametro1.data

      #calcular media
      for parametro2, parametro_media in zip(soma_parametros.parameters(), media.parameters()):
        parametro_media.data = parametro2.data/len(workers_maliciosos)

      # calculo desvio padrao
      for malicioso in workers_maliciosos:
        for parametro1, parametro2, parametro_media in zip(model[malicioso].parameters(), desvio_padrao_aux.parameters(), media.parameters()):
            diff_squared = (parametro1.data - parametro_media.data) ** 2
            parametro2.data += diff_squared  # Acumula diferença ao quadrado

      for parametro2, desvio in zip(desvio_padrao_aux.parameters(), desvio_padrao.parameters()):
        desvio.data = torch.sqrt(parametro2.data / len(workers_maliciosos))

      # alterar parametros
      for malicioso in workers_maliciosos:
        #print("inicio da iteracao")
        for parametro, desvio, parametro_media in zip(model[malicioso].parameters(), desvio_padrao.parameters(), media.parameters()):
          #print("parametro")
          #print(parametro.data)
          #print("desvio")
          #print(desvio.data)
          parametro.data = parametro_media.data + z * desvio.data
          #print("parametro")
          #print(parametro.data)



    return 0


    print("inicio ataque:")


    model_dict = [model[i].state_dict() for i in workers_maliciosos]
    model_flattened = [torch.cat([param.view(-1) for param in model_dict[i].values()]) for i in workers_maliciosos]

    #calcular a taxa de perturbação
    s = total_clientes/2 + 1 - len(workers_maliciosos)
    possibilidade = (total_clientes-s)/total_clientes
    z = norm.ppf(possibilidade)

    media_pesos = []
    media_dimensao=[]

    for i, _ in enumerate(model[0].children()):
      media_pesos.append(0)



    for w in [0]:
      for i, layer in enumerate(model[w].children()):
          if hasattr(layer, 'weight'):
            pesos=layer.weight.data
            print("pesos")
            print(pesos)



    print("media_dimensao")
    print(media_dimensao)

    print("\nmodel_dict\n")
    print(model_dict)
    print("\nmodel dict[0]\n")
    print(model_dict[0])

    print("\nflatend:")
    print(model_flattened)
    print("\noutro 1\n\n")
    print(model_flattened[0].shape)
    print("\noutro 2\n\n")
    print(model_flattened[0])


    with torch.no_grad():
      for w in workers_maliciosos:
        for i, layer in enumerate(model[w].children()):
          if hasattr(layer, 'weight'):
            #layer.weight.data = media_pesos[i] + z * desvio_padrao[i]
            print(" ")




    return 0




In [ ]:
def simular(wm):
    model = []              # armazenará instacias de modelos de neural networks
    optimizer = []          # armazenará optimizadores para cada instancia de modelo
    loss_fn = []            # armazena as funções de perda para cada instancia de modelo

    workers_maliciosos = []
    for i in range(0, wm):
        workers_maliciosos.append(i)

    for i in range(0, args.n_workers):
        model.append(Net().to(device))
        optimizer.append(optim.SGD(model[i].parameters(), lr=args.learning_rate))
        loss_fn.append(F.nll_loss)

    #cria args.n_workers trainloaders e um testloader geral para testar o modelo global
    trainloaders, testloader = prepare_dataset()

    global_model = Net().to(device)

    for i in range(1, args.epochs+1):
        #treina o modelo local de cada worker
        train(args, model, device, trainloaders, optimizer, loss_fn, i)

        if wm>1:
            ataque(args, global_model, model, workers_maliciosos)

        if i % 4 == 0:
            #agrega os modelos locais no modelo global
            #os modelos locais são atualizados para o modelo global
            fedavg_aggregation(model, global_model)

            #testa o modelo global
            test(global_model, device, testloader)

for wm in [1, 2, 3, 4, 5, 6, 7]:          # cada número na lista representa a quantidade de atacantes maliciosos e cada iteração do for é uma simulação
  print(f"Quantidade de workers maliciosos: {wm}")
  with open(f"teste{wm}.txt", 'w') as file:
    file.write(f"Quantidade de workers maliciosos: {wm}\n")
  simular(wm)


Quantidade de workers maliciosos: 1
Epoch 1 | Worker: 0 | TRAIN: Last Loss: 2.26885
Epoch 1 | Worker: 1 | TRAIN: Last Loss: 2.25666
Epoch 1 | Worker: 2 | TRAIN: Last Loss: 2.24423
Epoch 1 | Worker: 3 | TRAIN: Last Loss: 2.27278
Epoch 1 | Worker: 4 | TRAIN: Last Loss: 2.26814
Epoch 1 | Worker: 5 | TRAIN: Last Loss: 2.26743
Epoch 1 | Worker: 6 | TRAIN: Last Loss: 2.27936
Epoch 1 | Worker: 7 | TRAIN: Last Loss: 2.26334
Epoch 1 | Worker: 8 | TRAIN: Last Loss: 2.25437
Epoch 1 | Worker: 9 | TRAIN: Last Loss: 2.28967
Epoch 2 | Worker: 0 | TRAIN: Last Loss: 2.18270
Epoch 2 | Worker: 1 | TRAIN: Last Loss: 2.10908
Epoch 2 | Worker: 2 | TRAIN: Last Loss: 2.08641
Epoch 2 | Worker: 3 | TRAIN: Last Loss: 2.21701
Epoch 2 | Worker: 4 | TRAIN: Last Loss: 2.18877
Epoch 2 | Worker: 5 | TRAIN: Last Loss: 2.18278
Epoch 2 | Worker: 6 | TRAIN: Last Loss: 2.23254
Epoch 2 | Worker: 7 | TRAIN: Last Loss: 2.17366
Epoch 2 | Worker: 8 | TRAIN: Last Loss: 2.15470
Epoch 2 | Worker: 9 | TRAIN: Last Loss: 2.27132
Epoc